In [1]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging

from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json

In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("ProximityTree") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

config_DataIngestion = {
            # "data_path": project_root + "/ECG5000/*.tsv" # IF WE DECIDE TO USE .tsv FILES
            "data_path": project_root + "/fulldataset_ECG5000.csv", # IF WE DECIDE TO USE .csv FILES
            "data_percentage": 0.5  #*100% Percentage of data to load for SW development
            }   

ingestion = DataIngestion(spark,config_DataIngestion)
df = ingestion.load_data()
print(df.show(5))
df.printSchema()

Data Path is c:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\BigDataProject_repo/fulldataset_ECG5000.csv and loading 50.0% of data ++++++++++++++++++++++

Data size is :2527

+-----+-------------+-----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-------------+------------+------------+------------+------------+------------+------------+-----------+------------+-----------+------------+------------+-------------+-----------+------------+------------+------------+-----------+-------------+-----------+----------+---

In [4]:
config_preproc = {}    
preprocessor = Preprocessor(config_preproc)

preprocessed_df = preprocessor.run_preprocessing(df)
print(preprocessed_df.show(5))

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+----

In [5]:
df.count()

2527

In [6]:
num_partitions = df.rdd.getNumPartitions() # Number of partitions in the DataFrame
num_partitions

2

In [7]:
for partition in range(df.rdd.getNumPartitions()):
    print(f"Number of rows in partition {partition+1}: {len(df.rdd.glom().collect()[partition])}")
    

Number of rows in partition 1: 1338
Number of rows in partition 2: 1189


In [8]:
n_splitters = 5

fraction = n_splitters / df.count()  # Fraction of data to sample for each splitter
fraction

0.0019786307874950534

In [11]:
splitters = df.rdd.glom().sample(fraction=0.25, withReplacement=False).collect()
splitters

[]

In [33]:
from random import sample

def sample_from_partition(index, iterator):
    if index==0:
        partition_data = list(iterator)
        # Randomly sample 5 values (or fewer if the partition has less than 5 rows)
        return sample(partition_data, min(5, len(partition_data)))
    else:
        return []
    
# def sample_from_partition(index, iterator):
#     for index in range(num_partitions):
#         partition_data = list(iterator)
#         # Randomly sample 5 values (or fewer if the partition has less than 5 rows)
#         return sample(partition_data, min(5, len(partition_data)))
    
splitters = df.rdd.mapPartitionsWithIndex(sample_from_partition) # stays within the partition
collected_splitters = splitters.collect() # collects to the driver
print("Randomly sampled data from partition 1:", collected_splitters)

len(collected_splitters)  # Number of sampled rows

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.nio.file.NoSuchFileException: C:\Users\benat\AppData\Local\Temp\blockmgr-4cd61848-cc87-44e7-bfc5-f0d12f72a4d6\1c
java.nio.file.NoSuchFileException: C:\Users\benat\AppData\Local\Temp\blockmgr-4cd61848-cc87-44e7-bfc5-f0d12f72a4d6\1c
	at java.base/sun.nio.fs.WindowsException.translateToIOException(WindowsException.java:85)
	at java.base/sun.nio.fs.WindowsException.rethrowAsIOException(WindowsException.java:103)
	at java.base/sun.nio.fs.WindowsException.rethrowAsIOException(WindowsException.java:108)
	at java.base/sun.nio.fs.WindowsFileSystemProvider.createDirectory(WindowsFileSystemProvider.java:521)
	at java.base/java.nio.file.Files.createDirectory(Files.java:700)
	at org.apache.spark.storage.DiskBlockManager.getFile(DiskBlockManager.scala:108)
	at org.apache.spark.storage.DiskSt

In [32]:
from dtaidistance import dtw

ts1 = collected_splitters[0]
ts2 = collected_splitters[0]

dtw.distance(ts1, ts2)  # Calculate the DTW distance between the two time series

0.0

In [31]:
from random import sample
from dtaidistance import dtw
from pyspark.sql import Row

def calculate_gini_impurity(split, partition_data):
    """
    Calculate the Gini impurity reduction for a given split.
    """
    # Split the data into two groups based on the split condition
    left_split = [row for row in partition_data if row['distance'] <= split]
    right_split = [row for row in partition_data if row['distance'] > split]
    
    # Calculate sizes of splits
    total_size = len(partition_data)
    left_size = len(left_split)
    right_size = len(right_split)
    
    # Calculate Gini impurity for each split
    def gini_impurity(split):
        if len(split) == 0:
            return 0
        labels = [row['label'] for row in split]
        label_counts = {label: labels.count(label) for label in set(labels)}
        impurity = 1 - sum((count / len(split)) ** 2 for count in label_counts.values())
        return impurity
    
    gini_left = gini_impurity(left_split)
    gini_right = gini_impurity(right_split)
    
    # Weighted Gini impurity
    weighted_gini = (left_size / total_size) * gini_left + (right_size / total_size) * gini_right
    return weighted_gini

def process_partition(index, iterator):
    """
    Process each partition to:
    1. Select 5 splitters.
    2. Calculate DTW distances.
    3. Calculate Gini impurity reduction for each splitter.
    """
    partition_data = list(iterator)
    if len(partition_data) == 0:
        return []
    
    # Randomly sample 5 splitters
    candidate_splitters = sample(partition_data, min(5, len(partition_data)))
    
    results = []
    for splitter in candidate_splitters:
        # Calculate DTW distance between the splitter and all other instances
        for row in partition_data:
            row['distance'] = dtw.distance(splitter['feature'], row['feature'])
        
        # Calculate Gini impurity reduction for the splitter
        gini_reduction = calculate_gini_impurity(splitter['distance'], partition_data)
        results.append((splitter, gini_reduction))
    
    return results

# Apply the function to each partition
splitter_gini_rdd = df.rdd.mapPartitionsWithIndex(process_partition)

# Collect results to the driver
splitter_gini_results = splitter_gini_rdd.collect()

# Select the splitter with the largest Gini impurity reduction
best_splitter = max(splitter_gini_results, key=lambda x: x[1])
print("Best splitter:", best_splitter)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.nio.file.NoSuchFileException: C:\Users\benat\AppData\Local\Temp\blockmgr-4cd61848-cc87-44e7-bfc5-f0d12f72a4d6\1a
java.nio.file.NoSuchFileException: C:\Users\benat\AppData\Local\Temp\blockmgr-4cd61848-cc87-44e7-bfc5-f0d12f72a4d6\1a
	at java.base/sun.nio.fs.WindowsException.translateToIOException(WindowsException.java:85)
	at java.base/sun.nio.fs.WindowsException.rethrowAsIOException(WindowsException.java:103)
	at java.base/sun.nio.fs.WindowsException.rethrowAsIOException(WindowsException.java:108)
	at java.base/sun.nio.fs.WindowsFileSystemProvider.createDirectory(WindowsFileSystemProvider.java:521)
	at java.base/java.nio.file.Files.createDirectory(Files.java:700)
	at org.apache.spark.storage.DiskBlockManager.getFile(DiskBlockManager.scala:108)
	at org.apache.spark.storage.DiskBl